In [5]:
import json
import numpy as np
import pandas as pd
import os

In [6]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

/home/subbainria/anaconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [12]:
data = pd.read_csv('./label.csv')

In [13]:
data.head()

,Imageid,Imagelabel,Comment,Comment label,Combined label
0,1.jpg,0,Someone who can't rise high in life always loo...,0,1
1,2.jpg,0,"If you are a model you have to fit the size,ot...",0,1
2,3.jpg,0,Policemen always bark.,1,1
3,4.jpg,0,Size and appearance don't matter what matter i...,0,1
4,5.jpg,0,Never worry about washing out pimples from you...,0,1


In [14]:
data.shape

(2049, 5)

In [16]:
img_feat = []
text_feat = []
for i in np.arange(data.shape[0]):
    try:
        image = preprocess(Image.open('./Images/'+data['Imageid'][i])).unsqueeze(0).to(device)
        text = clip.tokenize(data['Comment'][i]).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            img_feat.append(image_features.detach().numpy())
            text_feat.append(text_features.detach().numpy())
    except:
        print(i)
        continue

506
2040


In [17]:
vision_embeddings = np.array(img_feat)
print(vision_embeddings.shape)
text_embeddings = np.array(text_feat)
print(text_embeddings.shape)

(2047, 1, 512)
(2047, 1, 512)


In [18]:
np.save('img_feat_clip',np.reshape(vision_embeddings,(vision_embeddings.shape[0],vision_embeddings.shape[2])))
np.save('text_feat_clip',np.reshape(text_embeddings,(text_embeddings.shape[0],text_embeddings.shape[2])))